<a href="https://colab.research.google.com/github/AnkitSingh10-hub/TinyLlama_PEFT/blob/main/LLMfinetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U torch transformers peft trl bitsandbytes accelerate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 813.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 797.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 141.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [7]:
import torch
import os
os.environ["WANDB_DISABLED"] = "true"
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset, load_dataset
from collections import defaultdict
import random

# 1. DATA ENGINEERING (Matches CV: "Balanced sampling strategy")
# ---------------------------------------------------------
ds = load_dataset('bitext/Bitext-travel-llm-chatbot-training-dataset', split="train")

random.seed(42)
intent_groups = defaultdict(list)
for record in ds:
    intent_groups[record["intent"]].append(record)

# Calculate samples needed per intent to ensure balance
total_desired_samples = 100
total_intents = len(intent_groups)
samples_per_intent = total_desired_samples // total_intents

balanced_subset = []
for intent, examples in intent_groups.items():
    # Sample without replacement up to the limit
    sampled = random.sample(examples, min(samples_per_intent, len(examples)))
    balanced_subset.extend(sampled)

# Create dataset and format it
travel_chat_ds = Dataset.from_list(balanced_subset)

def format_conversation(row):
    # Matches CV: "Travel-intent recognition" context
    return {
        "text": f"Query: {row['instruction']}\nIntent: {row['intent']}\nResponse: {row['response']}"
    }

travel_chat_ds = travel_chat_ds.map(format_conversation)

# 2. MODEL & QUANTIZATION (Matches CV: "Quantization" & "TinyLlama")
# ---------------------------------------------------------
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v0.1"

# Quantization Config (This was missing in your original code)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, # Key addition for CV accuracy
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model.config.use_cache = False # Silence warnings for training
model = prepare_model_for_kbit_training(model)

# 3. PEFT CONFIGURATION (Matches CV: "LoRA", "Low-rank adaptation")
# ---------------------------------------------------------
lora_config = LoraConfig(
    r=16, # Slightly higher rank often helps with 1.1B models
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'v_proj'] # TinyLlama specific target modules
)

# 4. TRAINING (Matches CV: "Custom SFT Pipeline")
# ---------------------------------------------------------
sft_config = SFTConfig(
    max_steps=60, # Increased slightly for demo purposes
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    learning_rate=2e-4,
    fp16=True, # Use fp16 for faster training on GPU
    logging_steps=1,
    output_dir="./results",
    dataset_text_field="text" # Pointing to the formatted column
)

trainer = SFTTrainer(
    model=model,
    train_dataset=travel_chat_ds,
    peft_config=lora_config,
    args=sft_config
)

print("Starting Training...")
trainer.train()

# 5. INFERENCE (Matches CV: "Travel-intent recognition")
# ---------------------------------------------------------
print("\n--- Inference ---")
query = "Query: I need to buy a ticket to Kathmandu"
# Ensure inputs are moved to the same device as the model (GPU)
inputs = tokenizer(query, return_tensors="pt").to(model.device)

# Fix: specific max_new_tokens, removed conflicting max_length
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    temperature=0.7
)

decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Adding EOS to train dataset:   0%|          | 0/99 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/99 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/99 [00:00<?, ? examples/s]

...

Step,Training Loss
1,2.573300
2,2.521200
3,2.504200
4,2.349800
5,2.412100
6,2.428400
7,2.428000
8,2.293200
9,2.441400
10,2.152700


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).



--- Inference ---


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Query: I need to buy a ticket to Kathmandu, ma garfanglo Ap Ot War, who continued the side spread ground with red sn sn sn caught the prem covered last pow su swing inscipate spread techn Apellett garcrier al plane quickly snoked that spread, of all the late
